# Q&A over LangChain Docs

Let's evaluate your architecture on a Q&A dataset for the LangChain python docs. For more examples of how to test different embeddings, indexing strategies, and architectures, see the [Evaluating RAG Architectures on Benchmark Tasks](rag_evaluations.ipynb) notebook.

## Pre-requisites

We will install quite a few prerequisites for this example since we are comparing many techniques and models.

We will be using LangSmith to capture the evaluation traces. You can make a free account at [smith.langchain.com](https://smith.langchain.com/). Once you've done so, you can make an API key and set it below.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# %pip install -U --quiet langchain langsmith langchainhub langchain_benchmarks
# %pip install --quiet chromadb openai huggingface pandas langchain_experimental sentence_transformers pyarrow anthropic tiktoken

For this code to work, please configure LangSmith environment variables with your credentials.

In [ ]:
import os

# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "sk-..."  # Your API key

# # Silence warnings from HuggingFace
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Review Q&A Tasks

The registry provides configurations to test out common architectures on curated datasets.

In [2]:
from langchain_benchmarks import clone_public_dataset, registry

In [3]:
registry = registry.filter(Type="RetrievalTask")
registry

Name,Type,Dataset ID,Description
LangChain Docs Q&A,RetrievalTask,452ccafc-18e1-4314-885b-edd735f17b9d,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Semi-structured Earnings,RetrievalTask,c47d9617-ab99-4d6e-a6e6-92b8daf85a7d,Questions and answers based on PDFs containing tables and charts. The task provides the raw documents as well as factory methods to easily index them and create a retriever. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).


In [4]:
langchain_docs = registry["LangChain Docs Q&A"]
langchain_docs

Name,LangChain Docs Q&A
Type,RetrievalTask
Dataset ID,452ccafc-18e1-4314-885b-edd735f17b9d
Description,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Retriever Factories,"basic, parent-doc, hyde"
Architecture Factories,conversational-retrieval-qa
get_docs,<function load_cached_docs at 0x102d17240>


In [5]:
clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)

Dataset LangChain Docs Q&A already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/3f29798f-5939-4643-bd99-008ca66b72ed.


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

docs = langchain_docs.get_docs()
retriever_factory = langchain_docs.retriever_factories["basic"]
# Indexes the documents with the specified embeddings
# Note that this does not apply any chunking to the docs,
# which means the documents can be of arbitrary length
retriever = retriever_factory(embeddings, docs=docs)

0it [00:00, ?it/s]

In [30]:
from typing import Sequence
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain.chat_models import ChatAnthropic
from langchain.schema.document import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter


def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = (
            f"<document index='{i}'>\n"
            f"<source>{doc.metadata.get('source')}</source>\n"
               f"<doc_content>{doc.page_content}</doc_content>\n"
            "</document>"
        )
        formatted_docs.append(doc_string)
    formatted_str = "\n".join(formatted_docs)
    return f"<documents>\n{formatted_str}\n</documents>"



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant answering questions about LangChain."
        "\n{context}\n"
        "Respond solely based on the document content."),
        ("human", "{question}"),
    ]
)
llm = ChatAnthropic(model="claude-2.1", temperature=1)

response_generator = (prompt | llm | StrOutputParser()).with_config(
    run_name="GenerateResponse",
)
chain = (
        RunnableAssign(
            {
                "context": (
                    itemgetter("question") | retriever | format_docs
                ).with_config(run_name="FormatDocs")
            }
        )
        | response_generator
    )

In [33]:
chain.invoke({"question": "What's expression language?"})

' Based on the LangChain documentation, the LangChain Expression Language (LCEL) is a declarative way to easily compose chains together. Key things to know about LCEL:\n\n- It was designed to support putting prototypes into production with no code changes, from simple "prompt + LLM" chains to complex chains with hundreds of steps. \n\n- It provides streaming support - when you build chains with LCEL you get the best possible time-to-first-token, streaming tokens from the LLM to output parsers incrementally.\n\n- Chains built with LCEL can be called synchronously (like in a notebook) or asynchronously (like in a production server), using the same code.\n\n- LCEL automatically runs steps in parallel when possible, minimizing latency. \n\n- It supports configuring retries and fallbacks to make chains more reliable.\n\n- You can access intermediate chain results before the final output is ready, which helps with debugging and user notifications.\n\n- LCEL chains get input and output valida

### Evaluate

Let's evaluate your RAG architecture on the dataset now.

In [36]:
from langsmith.client import Client
from langchain_benchmarks.rag import get_eval_config

In [39]:
client = Client()
RAG_EVALUATION = get_eval_config()

test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=chain,
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-essential-pot-37' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/d57047ac-d6e7-49c2-bd52-e2158e2ce56f?eval=true

View all tests for Dataset LangChain Docs Q&A at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/3f29798f-5939-4643-bd99-008ca66b72ed
[>                                                 ] 1/86

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[------------------------------------------------->] 86/86
 Eval quantiles:
                               0.25      0.5      0.75      mean      mode
embedding_cosine_distance  0.086465  0.12291  0.159078  0.128270  0.046088
score_string:accuracy      0.500000  0.70000  1.000000  0.641860  0.700000
faithfulness               0.700000  1.00000  1.000000  0.860465  1.000000


In [ ]:
test_run.get_aggregate_feedback()

## Evaluate with a default factory

The task can define default chain and retriever "factories", whic provide a default architecture that you can modify by choosing the llms, prompts, etc. Let's try the `conversational-retrieval-qa` factory.

In [12]:
# Factory for creating a conversational retrieval QA chain
chain_factory = langchain_docs.architecture_factories["conversational-retrieval-qa"]

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = "sk-..."

In [16]:
from langchain.chat_models import ChatAnthropic

# Example
llm = ChatAnthropic(model="claude-2", temperature=1)


chain = chain_factory(retriever, llm=llm)

chain.invoke({"question": "What is expression language?"})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


' <b>\n\n- Expression language (LCEL) is a declarative way to easily compose chains together in LangChain. It was designed to support putting prototypes into production with no code changes, from simple "prompt + LLM" chains to complex chains with hundreds of steps [0].\n\n- Key features of LCEL include streaming support, asynchronous support, optimized parallel execution, configurable retries and fallbacks, access to intermediate results, input/output validation schemas, seamless integration with LangSmith tracing and LangServe deployment [0].\n\n- The LangChain cookbook contains examples of common tasks using LCEL like chaining prompts and LLMs, adding retrieval, querying databases, writing code, adding memory and moderation, etc [1].\n\n</b>\n\n[0] - https://langchain.org/docs/expression_language\n[1] - https://langchain.org/docs/expression_language/cookbook'

In [34]:
from functools import partial


test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, retriever=retriever, llm=llm),
    evaluation=RAG_EVALUATION,
    verbose=True,
)